# Data Cleaning

### Importing libraries 

In [1]:
import warnings
warnings.filterwarnings('ignore')

# pandas and numpy
import pandas as pd
import numpy as np

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib qt

from Functions import *

# sk learn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

# stats and scipy
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
import scipy.stats as stats

# optional code to display all values as floats to 4 decimals
pd.options.display.float_format = '{:,.4f}'.format

### Examining the raw dataset 

In [2]:
data = pd.read_csv(r'C:\Users\Nick\Documents\Flatiron Project 2\dsc-phase-2-project-online-master\dsc-phase-2-project-online-master\data\kc_house_data.csv')

In [3]:
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,"221,900.0000",3,1.0000,1180,5650,1.0000,nan,0.0000,...,7,1180,0.0,1955,0.0000,98178,47.5112,-122.2570,1340,5650
1,6414100192,12/9/2014,"538,000.0000",3,2.2500,2570,7242,2.0000,0.0000,0.0000,...,7,2170,400.0,1951,"1,991.0000",98125,47.7210,-122.3190,1690,7639
2,5631500400,2/25/2015,"180,000.0000",2,1.0000,770,10000,1.0000,0.0000,0.0000,...,6,770,0.0,1933,nan,98028,47.7379,-122.2330,2720,8062
3,2487200875,12/9/2014,"604,000.0000",4,3.0000,1960,5000,1.0000,0.0000,0.0000,...,7,1050,910.0,1965,0.0000,98136,47.5208,-122.3930,1360,5000
4,1954400510,2/18/2015,"510,000.0000",3,2.0000,1680,8080,1.0000,0.0000,0.0000,...,8,1680,0.0,1987,0.0000,98074,47.6168,-122.0450,1800,7503


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [5]:
data.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

### Cleaning nulls and removing erroneous data  

In [6]:
# cleans the sqft basement column, removing "?" and replacing with 0's

data['sqft_basement'] = np.where(data['sqft_living'] == data['sqft_above'], 0, data['sqft_basement'])
data['sqft_basement'] = data['sqft_basement'].replace("?", 0)

# calculates missing basement values by subtracting sqft_above from sqft_living

data['sqft_basement'] = np.where(data['sqft_living'] >= data['sqft_above'], data['sqft_living']-data['sqft_above'], data['sqft_basement'])

# removes row with 33 bedrooms

data = data[data['bedrooms'] != 33]

# saves cleaned dataframe to a csv

data.to_csv(r'C:\Users\Nick\Documents\Flatiron Project 2\dsc-phase-2-project-online-master\dsc-phase-2-project-online-master\data\cleaned_data.csv', index=False)

In [7]:
# create a new dataframe, dropping some categorical variables

data_ = data[['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','condition','grade','sqft_above','sqft_basement','sqft_living15','sqft_lot15']]

data_.columns = ['Price','Bedrooms','Bathrooms','sqft_House','sqft_Lot','Floors','Condition','Grade','sqft_Above_Ground','sqft_Basement','sqft_Nearby_Homes','sqft_Nearby_Lots']

data_ = data_.replace([np.inf, -np.inf, np.nan], 0).fillna(0)
data_.info()

# saves new dataframe as csv
data_.to_csv(r'C:\Users\Nick\Documents\Flatiron Project 2\dsc-phase-2-project-online-master\dsc-phase-2-project-online-master\data\cleaned_data_features.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21596 entries, 0 to 21596
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Price              21596 non-null  float64
 1   Bedrooms           21596 non-null  int64  
 2   Bathrooms          21596 non-null  float64
 3   sqft_House         21596 non-null  int64  
 4   sqft_Lot           21596 non-null  int64  
 5   Floors             21596 non-null  float64
 6   Condition          21596 non-null  int64  
 7   Grade              21596 non-null  int64  
 8   sqft_Above_Ground  21596 non-null  int64  
 9   sqft_Basement      21596 non-null  int64  
 10  sqft_Nearby_Homes  21596 non-null  int64  
 11  sqft_Nearby_Lots   21596 non-null  int64  
dtypes: float64(3), int64(9)
memory usage: 2.1 MB


### Normalize the dataframe by converting to z-scores

In [8]:
data_z = (data_ - data_.mean())/data_.std()
data_z = data_z.replace([np.inf, -np.inf, np.nan], 0).fillna(0)

# saves z-score dataframe as csv
data_z.to_csv(r'C:\Users\Nick\Documents\Flatiron Project 2\dsc-phase-2-project-online-master\dsc-phase-2-project-online-master\data\z_score_data.csv', index=False)

data_z.describe()

,Price,Bedrooms,Bathrooms,sqft_House,sqft_Lot,Floors,Condition,Grade,sqft_Above_Ground,sqft_Basement,sqft_Nearby_Homes,sqft_Nearby_Lots
count,"21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000"
mean,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000
std,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
min,-1.2584,-2.6234,-2.1012,-1.8629,-0.3521,-0.9156,-3.7046,-3.9702,-1.7138,-0.6590,-2.3170,-0.4439
25%,-0.5942,-0.4113,-0.4757,-0.7083,-0.2429,-0.9156,-0.6299,-0.5608,-0.7232,-0.6590,-0.7248,-0.2808
50%,-0.2458,-0.4113,0.1745,-0.1855,-0.1806,0.0109,-0.6299,-0.5608,-0.2762,-0.6590,-0.2140,-0.1884
75%,0.2850,0.6948,0.4996,0.5115,-0.1066,0.9374,0.9074,0.2916,0.5090,0.6061,0.5449,-0.0981
max,19.4888,8.4372,7.6517,12.4816,39.5102,3.7168,2.4448,4.5533,9.2072,10.2294,6.1634,31.4735


### Normalize the dataframe using a log transform

In [9]:
data_floats = data_.astype(float)
data_log = np.log(data_floats)
data_log = data_log.replace([np.inf, -np.inf, np.nan], 0).fillna(0)

# saves log transformed dataframe as csv
data_log.to_csv(r'C:\Users\Nick\Documents\Flatiron Project 2\dsc-phase-2-project-online-master\dsc-phase-2-project-online-master\data\log_transform_data.csv', index=False)

data_log.describe()

,Price,Bedrooms,Bathrooms,sqft_House,sqft_Lot,Floors,Condition,Grade,sqft_Above_Ground,sqft_Basement,sqft_Nearby_Homes,sqft_Nearby_Lots
count,"21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000","21,596.0000"
mean,13.0482,1.1777,0.6776,7.5507,8.9898,0.3385,1.2098,2.0245,7.3952,2.5302,7.5395,8.9608
std,0.5266,0.2827,0.3925,0.4242,0.9021,0.3523,0.1813,0.1493,0.4271,3.1702,0.3275,0.8126
min,11.2645,0.0000,-0.6931,5.9135,6.2538,0.0000,0.0000,1.0986,5.9135,0.0000,5.9890,6.4785
25%,12.6823,1.0986,0.5596,7.2654,8.5252,0.0000,1.0986,1.9459,7.0817,0.0000,7.3065,8.5370
50%,13.0170,1.0986,0.8109,7.5549,8.9384,0.4055,1.0986,1.9459,7.3524,0.0000,7.5175,8.9385
75%,13.3770,1.3863,0.9163,7.8438,9.2766,0.6931,1.3863,2.0794,7.7007,6.3279,7.7664,9.2186
max,15.8567,2.3979,2.0794,9.5134,14.3171,1.2528,1.6094,2.5649,9.1495,8.4805,8.7339,13.6776
